In [21]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [22]:
# !pip install dash
# !pip install dash_html_components
# !pip install dash_core_components
# !pip install dash.dependencies
# !pip install --upgrade dash dash-core-components dash-html-components dash-renderer
#! pip install dash_bootstrap_components

In [23]:
import dash
import dash_html_components as html #html components
import dash_core_components as dcc #built the layout
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash import html,dcc
import dash_table

In [24]:
#load the dataset
solar_df = pd.read_csv('data/Solar Data.csv',delimiter=',')

In [25]:
def merge_data(file1,file2, file3):
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    df3 = pd.read_csv(file3)

    # Merge the DataFrames
    merged_df = pd.merge(df1, df2, on=['Unix timestamp','STD Date Time'], how='inner')
    merged_df = pd.merge(merged_df, df3, on=['Unix timestamp','STD Date Time'], how='inner')

    merged_df = merged_df.rename(columns={'Quantity_x': 'Cafe', 'Quantity_y': 'Corridor', 'Quantity': 'E Heat EEC'})

    return merged_df

consumption = merge_data('data/Cafe.csv','data/Corridor.csv', 'data/E Heat EEC main Bulding.csv')

consumption


,Unix timestamp,STD Date Time,Cafe,Corridor,E Heat EEC
0,"1,672,531,200",1/1/2023 0:00,0.0,0.0,0.06
1,"1,672,532,100",1/1/2023 0:15,0.0,0.0,0.06
2,"1,672,533,000",1/1/2023 0:30,0.0,0.0,0.06
3,"1,672,533,900",1/1/2023 0:45,0.0,0.0,0.06
4,"1,672,534,800",1/1/2023 1:00,0.0,0.0,0.06
...,...,...,...,...,...
8635,"1,680,302,700",3/31/2023 22:45,0.0,0.0,0.06
8636,"1,680,303,600",3/31/2023 23:00,0.0,0.0,0.06
8637,"1,680,304,500",3/31/2023 23:15,0.0,0.0,0.06
8638,"1,680,305,400",3/31/2023 23:30,0.0,0.0,0.06


In [26]:
# Convert 'Date' column to datetime type
consumption['Date'] = pd.to_datetime(consumption['STD Date Time'], format='%m/%d/%Y %H:%M')
# Extract year from 'Date' column
consumption['Year'] = consumption['Date'].dt.year
# Extract year from 'Month' column
consumption['Month'] = consumption['Date'].dt.strftime('%b')
# Extract year and month, and format month as three-letter abbreviation
consumption['Year_Month'] = consumption['Date'].dt.strftime('%b-%Y')

consumption_monthly = consumption.groupby(['Year', 'Month', 'Year_Month']).agg({'Cafe': 'sum','Corridor': 'sum','E Heat EEC': 'sum'}).reset_index()
consumption_monthly

,Year,Month,Year_Month,Cafe,Corridor,E Heat EEC
0,2023,Feb,Feb-2023,416.64,188.16,631.68
1,2023,Jan,Jan-2023,461.28,208.32,699.36
2,2023,Mar,Mar-2023,461.28,208.32,699.36


In [27]:
# Set up the bar graph
fig_consumption = go.Figure()

# Add the bar trace for Machine A
fig_consumption.add_trace(go.Bar(x=consumption_monthly['Year_Month'],y=consumption_monthly['Cafe'],name='Cafe'))

# Add the bar trace for Machine B
fig_consumption.add_trace(go.Bar(x=consumption_monthly['Year_Month'],y=consumption_monthly['Corridor'],name='Corridor'))

# Add the bar trace for Machine C
fig_consumption.add_trace(go.Bar(x=consumption_monthly['Year_Month'],y=consumption_monthly['E Heat EEC'],name='E Heat EEC'))

# Update the layout
fig_consumption.update_layout(
    title='Monthly Area Wise Consumption',
    xaxis_title='Month',
    yaxis_title='Consumption',
    barmode='group'
)
# Define the desired order of categories
category_order = ['Jan-2023', 'Feb-2023', 'Mar-2023']

# Update the x-axis category order
fig_consumption.update_xaxes(categoryorder='array', categoryarray=category_order)

# Show the bar graph
fig_consumption.show()

In [28]:
# Convert 'Date' column to datetime type
solar_df['Date'] = pd.to_datetime(solar_df['Time'], format='%m/%d/%Y %H:%M')
# Extract year from 'Date' column
solar_df['Year'] = solar_df['Date'].dt.year
# Extract year from 'Month' column
solar_df['Month'] = solar_df['Date'].dt.strftime('%b')
# Extract year and month, and format month as three-letter abbreviation
solar_df['Year_Month'] = solar_df['Date'].dt.strftime('%b-%Y')
solar_df.head()

,Time,gen,Date,Year,Month,Year_Month
0,1/1/2018 0:00,23.362676,2018-01-01,2018,Jan,Jan-2018
1,1/2/2018 0:00,18.812548,2018-01-02,2018,Jan,Jan-2018
2,1/3/2018 0:00,29.057803,2018-01-03,2018,Jan,Jan-2018
3,1/4/2018 0:00,22.020129,2018-01-04,2018,Jan,Jan-2018
4,1/5/2018 0:00,29.631941,2018-01-05,2018,Jan,Jan-2018


In [29]:
# Group the data by year and month and calculate the sum of generation
summary_df = solar_df.groupby(['Year', 'Month']).agg({'gen': 'sum'}).reset_index()
year_summary_df = solar_df.groupby(['Year']).agg({'gen': 'sum'}).reset_index()
# Print the summary DataFrame
print(summary_df)

    Year Month          gen
0   2018   Apr  3133.815635
1   2018   Aug  3964.456831
2   2018   Dec   549.544245
3   2018   Feb  1242.175692
4   2018   Jan   814.565718
..   ...   ...          ...
61  2023   Feb   653.629156
62  2023   Jan   377.074682
63  2023   Jun   536.636977
64  2023   Mar  1058.381149
65  2023   May  2749.092302

[66 rows x 3 columns]


In [30]:

solar_monthly = solar_df.groupby(['Year', 'Month', 'Year_Month']).agg({'gen': 'sum'}).reset_index()

#Merging the consumption and generation data set.
consumtion_generation = pd.merge(solar_monthly, consumption_monthly, on=['Year_Month','Year','Month'], how='inner')

# Sum up the values in the three columns and add as a new column
consumtion_generation['TTL Sum'] = consumtion_generation[['Cafe', 'Corridor', 'E Heat EEC']].sum(axis=1)
consumtion_generation

,Year,Month,Year_Month,gen,Cafe,Corridor,E Heat EEC,TTL Sum
0,2023,Feb,Feb-2023,653.629156,416.64,188.16,631.68,1236.48
1,2023,Jan,Jan-2023,377.074682,461.28,208.32,699.36,1368.96
2,2023,Mar,Mar-2023,1058.381149,461.28,208.32,699.36,1368.96


In [31]:
# Set up the bar graph
fig_consumption_genaration = go.Figure()

# Add the bar trace for Machine A
fig_consumption_genaration.add_trace(go.Bar(x=consumtion_generation['Year_Month'],y=consumtion_generation['TTL Sum'],name='TTL Power Consumption'))

# Add the bar trace for Machine A
fig_consumption_genaration.add_trace(go.Bar(x=consumtion_generation['Year_Month'],y=consumtion_generation['gen'],name='Solar Power Generation'))

# Update the layout
fig_consumption_genaration.update_layout(
    title='Monthly Area Wise Solar Genration Vs Solar Consumption (kw)',
    xaxis_title='Month',
    yaxis_title='Power Generation and Consumption (kw)',
    barmode='group'
)

# Define the desired order of categories
category_order = ['Jan-2023', 'Feb-2023', 'Mar-2023']

# Update the x-axis category order
fig_consumption_genaration.update_xaxes(categoryorder='array', categoryarray=category_order)

# Show the bar graph
fig_consumption_genaration.show()

In [32]:
solar_line = px.line (solar_df,x='Time',y='gen',title = f'Solar Generation')
solar_line.update_layout(plot_bgcolor='#5DADE2')
solar_line.update_traces(line_color='black')
solar_line

In [33]:
lineChart = px.line (solar_df,x='Date',y='gen',title = f'Solar Generation')

lineChart.update_layout(plot_bgcolor='#5DADE2')
lineChart.update_traces(line_color='#E74C3C')
# Define the seasons and their respective colors
seasons = [
    {'name': 'Winter', 'start': (12, 21), 'end': (3, 20), 'color': 'blue'},
    {'name': 'Spring', 'start': (3, 21), 'end': (6, 20), 'color': 'green'},
    {'name': 'Summer', 'start': (6, 21), 'end': (9, 22), 'color': 'yellow'},
    {'name': 'Autumn', 'start': (9, 23), 'end': (12, 20), 'color': 'orange'},
]

# Generate seasons for 5 years (2018-2022)
for year in range(2018, 2023):
    for season in seasons:
        start_date = pd.to_datetime(f"{year}-{season['start'][0]:02d}-{season['start'][1]:02d}")
        end_date = pd.to_datetime(f"{year}-{season['end'][0]:02d}-{season['end'][1]:02d}")
        lineChart.add_vrect(
            x0=start_date, x1=end_date,
            fillcolor=season['color'], opacity=0.2,
            layer='below', line_width=0
        )

# Show the line chart with highlighted seasons
lineChart.show()

In [34]:
app = dash.Dash(__name__, use_pages=True, external_stylesheets=[dbc.themes.SPACELAB])

In [35]:
# dash.register_page(__name__, path='/')

In [36]:
sidebar = dbc.Nav(
    [
        dbc.NavLink(
            [
                html.Div(page["name"], className="ms-2"),

            ],
            herf = page ["path"],
            active = "exact",
        )
        for page in dash.page_registry.values()
    ],
    vertical = True,
    pills = True,
    className = "big-light",

)

In [37]:
app.layout = html.Div(
    style={'backgroundColor': '#0B5345'},
    children = [ html.H1( 'BRISTOL COMMUNITY CENTER',
            style = {
                'textAlign' : 'center',
                'color' : '#FFFFFF'
            }
            ),
    html.Div('Energy Use and Efficiency',
             style = {
                'textAlign' : 'center',
                'color' : '#00b3ca'
            }
            ),
    html.Hr(),
    
    #plot and table in same raw
    html.Div(children = [
            dcc.Graph(id='solar_gen1', figure = lineChart, 
                      style={'width': '100%', 'backgroundColor': '#30D17C'} ),
            html.Br(),
            html.Br(),

            dash_table.DataTable(
                    id='table',
                    columns=[{'name': col, 'id': col} for col in year_summary_df.columns],
                    data=year_summary_df.to_dict('records'),
                    style_table={'width': '25%'},
                    style_cell={'textAlign': 'center'},
                    style_header={
                        'backgroundColor': 'lightgray',
                        'fontWeight': 'bold'
                    }
                )
            ], className='raw'),

   html.Br(),
   html.Br(),

    dcc.Dropdown(id ='year-dropdown',
                 options=[{'label': i, 'value':i} for i in summary_df['Year'].unique()],value='2023'),
                 
    dcc.Graph(id='solar_gen')
])

In [38]:
# app.layout = dbc.Container([
#     dbc.Row([
#         dbc.Col(html.Div("BRISTOL COMMUNITY CENTER",
#                          style={'fontSize':50, 'textAlign':'center'})
#                 # html.Div('Energy Use and Efficiency',
#                 #         style = {'textAlign' : 'center','color' : '#00b3ca'}),
#             )
#     ]),

#     html.Hr(),

#     dbc.Row(
#         [
#             dbc.Col(
#                 [
#                     sidebar
#                 ], xs=4, sm=4, md=2, lg=2, xl=2, xxl=2),

#             dbc.Col(
#                 [
#                     dash.page_container
#                 ], xs=8, sm=8, md=10, lg=10, xl=10, xxl=10)
#         ]
#     )
# ], fluid=True)

In [39]:
#Set up the callback function
@app.callback(
    Output(component_id = 'solar_gen',component_property = 'figure'),
    Input(component_id = 'year-dropdown', component_property = 'value')
)

def update_graph1(selected_year):
    filtered_gen = solar_df[solar_df['Year'] == selected_year]
    line_fig1 = px.line (filtered_gen,x='Time',y='gen',title = f'Solar Generation')
    
    return line_fig1
    

In [40]:
#Run the local server
if __name__ == '__main__':
    app.run_server(port =8080)

OSError: Address 'http://127.0.0.1:8080' already in use.
    Try passing a different port to run_server.